In [1]:
# Practica Numero 1
# 1.1 Random Forest
# 1.2 Gradient Boosting Machine

# Desarrollar un marco teorico y practico para cada uno de los casos, usando el dataset de EMNIST. 
# Debe incluir una conclusion de valoracion de casos de uso y tiempo de entrenamiento

In [1]:
#Imports required
import os
import pyodbc
import numpy as np
from dotenv import load_dotenv
from mainHelper import emnistHelper
from mainHelper import DataLoader as dl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Binarizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


#Config from .env variable
dotenv_path = '.env'
load_dotenv(dotenv_path)
#Get variables
SERVER = os.environ['SQL_SERVER']
DATABASE = os.environ['DATABASE']
UID = os.environ['UID']
PWD = os.environ['PWD']
RUN_HELPER = os.environ['RUN_HELPER']

In [2]:
#Connect to the database
conn = pyodbc.connect('Driver={SQL Server};'
                      f'Server={SERVER};'
                      f'Database={DATABASE};'
                      f'UID={UID};'
                      f'PWD={PWD};')
cursor = conn.cursor()

In [3]:

# Check if table Letters exists
if not cursor.tables(table='Letters', tableType='TABLE').fetchone():
    # Create a table for letters
    cursor.execute("CREATE TABLE Letters ( a0 int )")
    conn.commit()
    for i in range(1,785):
        cursor.execute(f'''
                        ALTER TABLE Letters
                        ADD a{i} int;
                    ''')
        conn.commit()

        
# Check if table Digits exists
if not cursor.tables(table='Digits', tableType='TABLE').fetchone():
    # Create a table for Digits
    cursor.execute("CREATE TABLE Digits ( a0 int )")
    conn.commit()
    for i in range(1,785):
        cursor.execute(f'''
                        ALTER TABLE Digits
                        ADD a{i} int;
                    ''')
        conn.commit()
        

In [ ]:
print(RUN_HELPER)
if RUN_HELPER:
    emnistHelper.emnist_to_sql(cursor, conn)

In [4]:
loader = dl.Data()
digits = loader.loadData('Digits', conn)
letters = loader.loadData('Letters', conn)

c:\Users\DougC\OneDrive\Escritorio\Andres\src\mainHelper\DataLoader.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(f'SELECT * FROM {table}', conn)
c:\Users\DougC\OneDrive\Escritorio\Andres\src\mainHelper\DataLoader.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(f'SELECT * FROM {table}', conn)


In [14]:
# Get x and y from data
#Digits
x_digits = loader.getX(digits)
y_digits = loader.getY(digits)
#Letters
x_letters = loader.getX(letters)
y_letters = loader.getY(letters)

In [8]:
#First digits
x_train, x_test, y_train, y_test = train_test_split(x_digits, y_digits, test_size = 0.2, random_state = 0)

In [15]:
#Second letters
x_train, x_test, y_train, y_test = train_test_split(x_letters, y_letters, test_size = 0.2, random_state = 0)

In [7]:
# Binarize the data to improve classification performance
binarizer = Binarizer(threshold=0.707)
x_train_bin = binarizer.fit_transform(x_train)
x_test_bin = binarizer.transform(x_test)

# Perform PCA to reduce the dimensionality of the data
pca = PCA(n_components=0.95)
x_train_pca = pca.fit_transform(x_train_bin)
x_test_pca = pca.transform(x_test_bin)

In [19]:
# Check the range of labels in y_train
print(f"Minimum label in y_train: {np.amin(y_train)}")
print(f"Maximum label in y_train: {np.amax(y_train)}")
print(f"Minimum label in y_test: {np.amin(y_test)}")
print(f"Maximum label in y_test: {np.amax(y_test)}")


Minimum label in y_train: 0
Maximum label in y_train: 51
Minimum label in y_test: 0
Maximum label in y_test: 51


In [19]:
print(y_train)

[40 30 43 ... 34  8 18]


In [18]:
#Just for letters:
# Subtract 10 from the labels in y_train and y_test
y_train -= 10
y_test -= 10

In [19]:
# Try different values for n_estimators and evaluate performance
n_estimators_list = [10, 50, 100, 150, 200]
for n_estimators in n_estimators_list:
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf.fit(x_train_pca, y_train)
    accuracy = rf.score(x_test_pca, y_test)
    print(f"n_estimators={n_estimators}, accuracy={accuracy}")


n_estimators=10, accuracy=0.5178800034138431
n_estimators=50, accuracy=0.6486301954425194
n_estimators=100, accuracy=0.6709055218912691
n_estimators=150, accuracy=0.6748314414952633
n_estimators=200, accuracy=0.6777332081590851


In [20]:
# Try different values for random_state and evaluate performance
random_state_list = [42, 123, 456, 789, 999]
for random_state in random_state_list:
    rf = RandomForestClassifier(n_estimators=100, random_state=random_state)
    rf.fit(x_train_pca, y_train)
    accuracy = rf.score(x_test_pca, y_test)
    print(f"random_state={random_state}, accuracy={accuracy}")

random_state=42, accuracy=0.6709055218912691
random_state=123, accuracy=0.6676623709140564
random_state=456, accuracy=0.6699667150294444
random_state=789, accuracy=0.6673209866006657
random_state=999, accuracy=0.6668942562089272


In [25]:
#Now we will check the GBM
import xgboost as xgb

# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(x_train_pca, label=y_train)
dtest = xgb.DMatrix(x_test_pca, label=y_test)

# Define the hyperparameters for the XGBoost model
params = {'objective': 'multi:softmax',
          'num_class': len(set(y_train)),
          'eval_metric': 'merror',
          'max_depth': 5,
          'eta': 0.1,
          'subsample': 0.8,
          'colsample_bytree': 0.8,
          'seed': 42}

# Try different values for num_rounds and evaluate performance
num_rounds_list = [10, 50, 100, 150, 200]
for num_rounds in num_rounds_list:
    # Train the XGBoost model
    xgb_model = xgb.train(params, dtrain, num_rounds)

    # Make predictions on the test data using the trained model
    preds = xgb_model.predict(dtest)

    # Evaluate the performance of the XGBoost model on the test data
    accuracy = sum(y_test == preds) / len(y_test)
    print(f"num_rounds={num_rounds}, accuracy={accuracy}")

num_rounds=10, accuracy=0.6363559322033898
num_rounds=50, accuracy=0.7027118644067797
num_rounds=100, accuracy=0.7242372881355932


In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
inputs = keras.Input(shape=(784))
x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(52, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 rescaling_1 (Rescaling)     (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               100480    
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 52)                6708      
                                                                 
Total params: 123,700
Trainable params: 123,700
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=20)

Fit on NumPy data
Epoch 1/20
738/738 [==============================] - 6s 4ms/step - loss: 1.3336 - acc: 0.6348
Epoch 2/20
738/738 [==============================] - 3s 4ms/step - loss: 0.7975 - acc: 0.7587
Epoch 3/20
738/738 [==============================] - 3s 5ms/step - loss: 0.6714 - acc: 0.7890
Epoch 4/20
738/738 [==============================] - 3s 4ms/step - loss: 0.6029 - acc: 0.8032
Epoch 5/20
738/738 [==============================] - 3s 4ms/step - loss: 0.5532 - acc: 0.8163
Epoch 6/20
738/738 [==============================] - 3s 4ms/step - loss: 0.5172 - acc: 0.8240
Epoch 7/20
738/738 [==============================] - 3s 4ms/step - loss: 0.4826 - acc: 0.8340
Epoch 8/20
738/738 [==============================] - 3s 4ms/step - loss: 0.4585 - acc: 0.8388
Epoch 9/20
738/738 [==============================] - 3s 4ms/step - loss: 0.4353 - acc: 0.8450
Epoch 10/20
738/738 [==============================] - 3s 5ms/step - loss: 0.4156 - acc: 0.8504
Epoch 11/20
738/738 [==========